# 570 Interim Deliverable 2 

## MIP formulation

#### Input data:  
$$\begin{aligned}
\text{Section information:}
&& I & = \text{{all course sections}} \\
&& student & = \text{{predicted number of students who will register for each section}} \\
&& PS & = \text{{number of sessions for each section}}  \\
&& TS & = \text{{the timelength of each session in hours (e.g., 1.5, 2, 3)}}  \\
&& I_2 & = \text{{sections with 2 sessions a week (i.e., MW and TH)}} \\
\\
\text{Classroom information:}
&& J & = \text{{classrooms}} \\
&& seat & = \text{{number of seats in each classroom}}  \\
\\
\text{Day-time information:}
&& P & = \text{{day of week (i.e., M T W H F)}}  \\
&& T & = \text{{start time of a section}} \\
\end{aligned}$$

#### Decision variable:
Let the binary decision variable $X_{ijpt}$ denotes whether we assign course section $i$ to classroom $j$ at day $p$ starting at time $t$.

#### Objective and constraints:
Our goal is to maximize the capacity utilization rate. The linear program is thus:
$$\begin{aligned}
\text{maximize} && \sum_{i\in I}\sum_{j\in J}\sum_{p\in P}\sum_{t\in T}{X_{ijpt}\frac{student_i}{seat_j}} \\
\text{subject to:} \\
\
\text{No start time conflict for classes:} && X_{ijpt} + X_{(I-i)jpt} & \le 1 && \forall i \in I, j \in J, p \in P, t \in T\\
\\
\text{No class conflict when a class is in session:} && X_{ijpt} + X_{i'jp(t+c)}& \le 1 && \forall i,i' \in I; j \in J; p \in P; t \in T; c \in \{1,2,...,TS_i*2-1\}\\
\\
\text{All required sessions must be scheduled:} && \sum_{j\in J}\sum_{p\in P}\sum_{t\in T}{X_{ijpt}} & = PS_i && \forall i \in I\\
\\
\text{Number of students must not exceed total seats:} && X_{ijpt} * student_i & \le seat_j && \forall i \in I, j \in J, p \in P, t \in T \\
\\
\text{No class scheduled  to last beyond school time (i.e., 9.30PM):} && X_{ijpt'} & = 0 && \forall i \in I, j \in J, p \in P, t' \in \{T_{max}-(TS_i*2)+2,...,T_{max}\} \\
\\
\text{Binding clause for courses with 2 sessions (MW):} && X_{ij'M't} & = X_{ij'W't} && \forall i \in I_2, j \in J, t \in T\\
\text{Binding clause for courses with 2 sessions (TH):} && X_{ij'T't} & = X_{ij'H't} && \forall i \in I_2, j \in J, t \in T\\
\\
\text{No Friday class for courses with 2 sessions:} && X_{ij'F't} & = 0 && \forall i \in I_2, j \in J, t \in T \\
\end{aligned}$$

# Python Code

In [1]:
import gurobipy as grb
import pandas as pd
import numpy as np
import datetime
from datetime import time
from datetime import timedelta
import math
import random

# for the room capacity file i remove the first record because it is same as the 7th record and will make some error in next step
capacity=pd.read_excel("Marshall_Room_Capacity_Chart.xlsx")
capacity1=capacity
course_enrollment=pd.read_excel("Marshall_Course_Enrollment_1516_1617.xlsx") # for student prediction
#course=pd.read_excel("Marshall_Course_Enrollment_1516_1617_small.xlsx")
#section_info=pd.read_excel("section information.xlsx")

In [2]:
course_enrollment.head()

,Course,Course Prefix,Course Suffix,Department,First Begin Time,First Days,First End Time,First Instructor,First Instructor UID,First Room,...,Second Begin Time,Second Days,Second End Time,Second Instructor,Second Instructor UID,Second Room,Section,Session,Term,Title
0,ACCT-370,ACCT,370,ACCT,10:00:00,F,11:50:00,"Hopkins, Merle, W",3.783354e+09,SLH200,...,NaN,NaN,NaN,NaN,NaN,NaN,14029,1,20153,External Financial Reporting Issues
1,ACCT-370,ACCT,370,ACCT,08:00:00,MW,09:50:00,"Hopkins, Merle, W",3.783354e+09,ACC303,...,NaN,NaN,NaN,NaN,NaN,NaN,14025,1,20153,External Financial Reporting Issues
2,ACCT-370,ACCT,370,ACCT,10:00:00,MW,11:50:00,"Hopkins, Merle, W",3.783354e+09,ACC303,...,NaN,NaN,NaN,NaN,NaN,NaN,14026,1,20153,External Financial Reporting Issues
3,ACCT-370,ACCT,370,ACCT,12:00:00,MW,13:50:00,"Hopkins, Merle, W",3.783354e+09,ACC303,...,NaN,NaN,NaN,NaN,NaN,NaN,14027,1,20153,External Financial Reporting Issues
4,ACCT-371,ACCT,371,ACCT,10:00:00,F,11:50:00,NaN,NaN,SLH200,...,NaN,NaN,NaN,NaN,NaN,NaN,14044,1,20153,Introduction to Accounting Systems


## Extra data from original dataset

In [3]:
# choose 20153 term as sample
data=course_enrollment.loc[course_enrollment.loc[:,"Term"]==20153,("Course","Section","First Days","First Begin Time","First End Time",
                                                                   "First Room","First Instructor","Reg Count","Seats")]
data.columns=["Course","Section","FirstDays","FirstBeginTime","FirstEndTime","FirstRoom","FirstInstructor","regcount","Seats"]
# we only use those M/T/W/H/F/MW/TH
data=data[(data.FirstDays=="M")|(data.FirstDays=="T")|(data.FirstDays=="W")|(data.FirstDays=="H")|(data.FirstDays=="F")|(data.FirstDays=="MW")|(data.FirstDays=="TH")]
data=data[(data.FirstRoom!="ONLINE")&(data.FirstRoom!="DEN@Viterbi")]
# remove the record that have nan in first begintime
data=data[(data.FirstBeginTime==data.FirstBeginTime)|(data.FirstEndTime==data.FirstEndTime)]
# remove the record that have nan in professor
data=data[data.FirstInstructor==data.FirstInstructor]
#data=data.loc[:,("Course","Section","FirstDays","FirstBeginTime","FirstEndTime","FirstInstructor","FirstRoom","regcount")]

In [4]:
# timelength
timelength=[]
for i in data.index:
    #print i
    #print data.loc[i,:]
    #print data.loc[i,:]
    time1=data.loc[i,"FirstEndTime"]
    #print type(time1)
    time2=data.loc[i,"FirstBeginTime"]
    minutediff=((time1.hour*60+time1.minute)-(time2.hour*60+time2.minute)+10)
    halfhour=minutediff/30
    if minutediff%30>0:
        halfhour+=1
    timelength.append(halfhour)

In [5]:
# pattern
pattern=[]
for i in data.index:
    day=data.loc[i,"FirstDays"]
    if (day=="M") or (day=="T") or (day=="W") or (day=="H") or (day=="F"):
        #print day
        pattern.append(1)
    elif (day=="MW") or (day=="TH"):
        #print day
        pattern.append(2)

In [6]:
# students' registeration prediction
student={}
for i in data["Section"]:
    sum1=sum(course_enrollment.loc[course_enrollment.loc[:,"Section"]==i,"Reg Count"])
    len1=len(course_enrollment.loc[course_enrollment.loc[:,"Section"]==i,"Reg Count"])
    student[i]=round(sum1/len1)

Use the "section_info=section_info[section_info.timelength<=2]" below to control the size of dataset (change value of 2), i only tried small one

In [7]:
# build section_info table
section_info=pd.DataFrame({"course":data["Course"],"section":data["Section"],"pattern":pattern,
                           "timelength":timelength,"FirstInstructor":data["FirstInstructor"],
                           "Reg Count":student.values()})
# we only use those timelength less than or equal to 4 hours
section_info=section_info[section_info.timelength<=8]
# build table of professor and sections
Prof=section_info["FirstInstructor"].unique()
professor={}
for i in Prof:
    professor[i]=section_info.loc[section_info.loc[:,"FirstInstructor"]==i,"section"]
print max(section_info["timelength"])
print len(section_info)
#print section_info.index
# for change the index, do not delete 
section_info.index=range(len(section_info))
# get how many sessions we have in this sechedule
numofsections=sum(section_info["pattern"])
numofprof=len(Prof)

8
514


### change the size of sample dataset

In [8]:
# select a subset of section id 
# only keep those classes with reg count less than 150, so ll105 can meet the need of seats
section_info=section_info.loc[section_info.loc[:,"Reg Count"]<=149,:]#77
section_info=section_info.reset_index(drop=True)
# randomly choose 100 sections
random.seed(1)
indexofsection=random.sample(section_info.index,85)
section_info=section_info.iloc[indexofsection,:]
section_info=section_info.reset_index(drop=True)
#section_info=section_info.iloc[range(30),:]
#reset dataframe's index 


Prof=section_info["FirstInstructor"].unique()
professor={}
for i in Prof:
    professor[i]=section_info.loc[section_info.loc[:,"FirstInstructor"]==i,"section"]
numofsections=sum(section_info["pattern"])
numofprof=len(Prof)
print numofsections
print numofprof

140
60


In [9]:
print section_info.iloc[range(50,max(section_info.index)+1),range(1,6)]



    Reg Count     course  pattern  section  timelength
50       31.0   BUAD-310        1    14917           3
51       47.0   BUAD-497        2    15108           4
52       30.0   ACCT-377        2    14065           4
53       17.0  BUAD-285A        2    14512           4
54       34.0   GSBA-528        2    15723           6
55       38.0   BUAD-306        2    14785           4
56       37.0   BAEP-460        1    14387           8
57       46.0  BUAD-311T        2    14968           4
58       19.0    FBE-441        2    15362           4
59       18.0    FBE-324        2    15343           4
60       69.0   BUAD-304        1    14725           4
61       30.0   BUAD-310        1    14892           3
62       34.0  GSBA-521B        2    15660           3
63       31.0   BUAD-310        1    14920           3
64       33.0   BUAD-304        1    14732           4
65       36.0   BUAD-310        1    14895           3
66       38.0   ACCT-410        2    14096           4
67       4

In [10]:
print section_info.iloc[range(50,max(section_info.index)+1),range(1,6)]


    Reg Count     course  pattern  section  timelength
50       31.0   BUAD-310        1    14917           3
51       47.0   BUAD-497        2    15108           4
52       30.0   ACCT-377        2    14065           4
53       17.0  BUAD-285A        2    14512           4
54       34.0   GSBA-528        2    15723           6
55       38.0   BUAD-306        2    14785           4
56       37.0   BAEP-460        1    14387           8
57       46.0  BUAD-311T        2    14968           4
58       19.0    FBE-441        2    15362           4
59       18.0    FBE-324        2    15343           4
60       69.0   BUAD-304        1    14725           4
61       30.0   BUAD-310        1    14892           3
62       34.0  GSBA-521B        2    15660           3
63       31.0   BUAD-310        1    14920           3
64       33.0   BUAD-304        1    14732           4
65       36.0   BUAD-310        1    14895           3
66       38.0   ACCT-410        2    14096           4
67       4

In [11]:
professor

{u'Ahern, Kenneth': 25    14786
 Name: section, dtype: int64, u'Allen, Eric': 26    14514
 28    14513
 Name: section, dtype: int64, u'Ansari, Arif': 6    16271
 Name: section, dtype: int64, u'Aritz, Jolanta': 38    66771
 Name: section, dtype: int64, u'Barth, Steve': 47    15099
 Name: section, dtype: int64, u'Bayiz, Murat': 84    15753
 Name: section, dtype: int64, u'Bemis, Nimfa, Abarquez': 24    14495
 30    14497
 Name: section, dtype: int64, u'Bonner, Sarah, Elizabeth': 44    16120
 Name: section, dtype: int64, u'Bresnahan, Chris': 23    14741
 Name: section, dtype: int64, u'Bristow, Duke': 15    15398
 81    15904
 Name: section, dtype: int64, u'Burgos, Miriam, T': 2     15561
 12    15560
 Name: section, dtype: int64, u'Cavanaugh, Lisa, Ann': 10    16474
 34    16470
 Name: section, dtype: int64, u'Cerling, Lee': 78    66769
 Name: section, dtype: int64, u'Colman, Maria': 46    14650
 Name: section, dtype: int64, u'Coombs, Michael, Wallace': 18    16671
 64    14732
 Name: sect

In [12]:
capacity1

,Room,Size
0,ACC201,48
1,ACC205,36
2,ACC236,39
3,ACC303,46
4,ACC306B,16
5,ACC310,54
6,ACC312,20
7,BRI202,42
8,BRI202A,34
9,BRI5,42


In [13]:
# as we just use a 100 sections from origional dataset, we try to manully choose available classroom to make the ratio of 
# of registed student and # of available seats similiar for origional and new subset data
course20153=course_enrollment.loc[course_enrollment.loc[:,"Term"]==20153,:]
course20153=course20153[(course20153.loc[:,"First End Time"]==course20153.loc[:,"First End Time"])&
                        (course20153.loc[:,"First Begin Time"]==course20153.loc[:,"First Begin Time"])]
course20153=course20153.loc[course20153.loc[:,"Reg Count"]<=149,:]


# calculate timelength for 20153
timelength=[]
for i in course20153.index:
    time1=course20153.loc[i,"First End Time"]
    time2=course20153.loc[i,"First Begin Time"]
    minutediff=((time1.hour*60+time1.minute)-(time2.hour*60+time2.minute)+10)
    halfhour=minutediff/30
    if minutediff%30>0:
        halfhour+=1
    timelength.append(halfhour)
course20153.loc[:,"timelength"]=timelength

# origional ratio of registered student and number of seats
ratio1=float(sum(course20153["Reg Count"]))/(sum(capacity1["Size"]))
print "\n"
print "no time "+str(ratio1)
# manully choose classroom to make the new ratio is similiar to the origional ratio
capacity=pd.DataFrame({"Room":["JFF LL105","JFF414","ACC201","JKP104","ACC312","BRI5"],
                       "Size":[149,60,48,56,20,42
                              ]})
#capacity=pd.DataFrame({"Room":["JKP110","HOH1","JFF414","ACC201","JKP104","BRI202"#"ACC201","ACC205","BRI202","JFF414",
 #                              ],
  #                     "Size":[77,73,60,48,56,42#48,36,42,73,60,
   #                           ]})

#capacity=pd.DataFrame({"Room":["JFF LL105"],"Size":[149]})
ratio2=sum(section_info["Reg Count"])/sum(capacity["Size"])
print "\n"
print "no time "+str(ratio2)



no time 9.69407894737


no time 8.4


In [14]:
#capacity=pd.DataFrame({"Room":["JFF LL105"],"Size":[149]})

## Build variables

In [15]:
# What we need to extract from data
# prediction about student registeration for every section
# how many section we need to schedule
# the pattern, time length of each section
# classroom capacity

# Variables
# course name
C={}
index=0
for i in section_info["section"]:
    C[i]=section_info.loc[section_info.loc[:,"section"]==i,"course"].get(index)
    index+=1
# section of course
I=section_info["section"]
# classroom
J=capacity["Room"]
# pattern of session (less than catagories in origional dataset)
P=["Monday","Tuesday","Wednesday","Thursday","Friday"]
P=pd.Series(i for i in P)
# start time of session (from 8:00am to 9:00pm,so the real classtime is between 8:00am to 10:00pm)
T1={}
time = datetime.timedelta(hours=8,minutes=0, seconds=0)
for i in range(27):
    T1[str(time+timedelta(hours=0.5*i))]=i
T=pd.Series(i for i in range(27))

# students' registeration prediction
student={}
for i in I:
    sum1=sum(course_enrollment.loc[course_enrollment.loc[:,"Section"]==i,"Reg Count"])
    len1=len(course_enrollment.loc[course_enrollment.loc[:,"Section"]==i,"Reg Count"])
    student[i]=round(sum1/len1)
#student=pd.Series(i for i in student)

# seat
seat={}
for index,row in capacity.iterrows():
    seat[row["Room"]]=row["Size"]
#seat=pd.Series(i for i in seat)
    
# pattern of each section
PS={}
index=0
for i in I:
    PS[i]=section_info.loc[section_info.loc[:,"section"]==i,"pattern"].get(index)
    index+=1
#PS=pd.Series(i for i in PS)
    
# timelength of each section
TS={}
index=0
for i in I:
    TS[i]=section_info.loc[section_info.loc[:,"section"]==i,"timelength"].get(index)
    index+=1
#TS=pd.Series(i for i in TS)

#define primetime
primetime=range(4,21)

In [16]:
capacity

,Room,Size
0,JFF LL105,149
1,JFF414,60
2,ACC201,48
3,JKP104,56
4,ACC312,20
5,BRI5,42


## Build Model

In [17]:
# Building model
mod=grb.Model()

## Decision Variables

In [18]:
X={}
for i in I: # section id
    for j in J: # classroom
        for p in P: # day of week
             for t in T: # start timeslot of sesstion
                    X[i,j,p,t]=mod.addVar(vtype=grb.GRB.BINARY, name='x[{0},{1},{2},{3}]'.format(i,j,p,t))

                    
# add one more variable PD[f,p,b], f is faculty(professor), p is day of week, b is the building
# PD[f,p,b]=0 means that prof f in p day do not have class in building b 
PD={}
Building=["ACC","BRI","HOH","JFF","JKP"]
for f in Prof:
    for p in P:
        for b in Building:
            PD[f,p,b]=mod.addVar(vtype=grb.GRB.BINARY,name='PD[{0},{1},{2}]'.format(f,p,b))

# add one more numeric variable Dp,f means that professor f have to show up in campus in p day of week
D={}
for f in Prof:
    for p in P:
        D[p,f]=mod.addVar(vtype=grb.GRB.BINARY,name='x[{0}]'.format(p))

#add one more varible PTi. PTi means that how many sessions of section i are in prime time
PT={}
for i in I:
    PT[i]=mod.addVar(vtype=grb.GRB.INTEGER,name='PT[{0}]'.format(i))


## Objective

In [19]:
# Objective
# As i set the constraint1 below, so the section with 2 or 3 sessions will have higher weight here
# but if i do not set constraint1, we cannot promise other sections will not take the vacancy
# for example, if DSO570 assigned to M/W 12:00, namely X[DSO570,j,"MW",12:00]=1, but X[i,j,"M",12:00]=0 and X[i,j,"W",12:00]=0, 
# so other courses maight be assign to this time too
mod.setObjective((
    # classroom utilization (max)
    3*sum(X[i,j,p,t]*student[i]/seat[j] for i in I for j in J for p in P for t in T)/numofsections
    # number of days that professor has class (min)
    -sum(D[p,f] for f in Prof for p in P)/numofprof
    # number of buildings that professor need to go in anyday that he has class (min
    #-sum(PD[f,p,b] for f in Prof for p in P for b in Building)/numofprof
    # prime time utilization (max)
    +2*sum(PT[i] for i in I)/numofsections
),sense=grb.GRB.MAXIMIZE)

## Constraints

In [20]:
# constraints
I_twoday=pd.Series()
I_oneday=pd.Series()
for i in I:
    if PS.get(i)==1:
        I_oneday=I_oneday.append(pd.Series(i))
    else:
        I_twoday=I_twoday.append(pd.Series(i))

In [21]:
# constraint1
# in each time slot in each day every professor can only teach one session at the same time
constraint1={}
for t in T:
    for p in P:
        for prof in Prof:
            constraint1[t,p,prof]=mod.addConstr(sum(X[i,j,p,t] for i in professor[prof].values for j in J)<=1)
            

In [22]:
# new constraint2
# every timeslot in every classroom in a specific day, there will be only one or zero class
#constraint2={}
#for j in J:
 #   for t in T:
  #      for p in P:
   #         constraint2[j,t,p]=mod.addConstr(sum(X[i,j,p,t] for i in I)<=1)
            
# in the next (number of required sessions)*2 timeslot all of X should be 0

# for i1 in I:
#     I2=I
#     I2=I2.drop(pd.Index(I).get_loc(i1))
#     for j in J:
#         for t in T:
#             for p in P:
#                 timeslots=int(TS[i1])
#                 if t<=(27-timeslots): # avoid a situation that three hours course is assigned to 8:00 pm
#                     #print i1,j,p,t
#                     constraint2[i1,j,t,p]=mod.addConstr((X[i1,j,p,t]+sum(X[i2,j,p,t+num] for i2 in I2 for num in range(0,timeslots)))<=1,name="Problem {0} {1} {2} {3}".format(i1,j,p,t))
                    
                    

In [23]:
y=mod.addVars(I,J,P,T,vtype=grb.GRB.BINARY)

for i in I:
    for j in J:
        for t in T:
            for p in P:
                timeslots=int(TS[i])
                mod.addConstr(y[i,j,p,t]==sum(X[i,j,p,t2] for t2 in range(max(0,t-timeslots+1),t+1)),name="constrain2_"+str(i))
#for i in I:
 #   for j in J:
  #      for t in T:
   #         for p in P:
    #            timeslots=int(TS[i])
     #           mod.addConstr(y[i,j,p,t]==sum(X[i,j,p,t2] for t2 in range(t,min(27,t+timeslots-1))))



for j in J:
    for t in T:
        for p in P:
            mod.addConstr(sum(y[i,j,p,t] for i in I)<=1)


In [24]:
# this constraint3 is only for those sections whose pattern is 2, which means they have two session every week.
constraint3={}
for i in I_twoday:
    for j in J:
        for t in T:
            constraint3[i,j,"M",t]=mod.addConstr(X[i,j,"Wednesday",t]== X[i,j,"Monday",t],name="") 
            # the section with two sessions must have same 0/1 in M and W
            
            constraint3[i,j,"T",t]=mod.addConstr(X[i,j,"Thursday",t] == X[i,j,"Tuesday",t],name="") 
            # the section with two sessions must have same 0/1 in M and W

In [25]:
# this constraint4 is to control that when we maximum the utilization rate,the number of student registered 
#for one section should less than the classroom capacity
constraint4={}
for i in I:
    for j in J:
        for p in P:
             for t in T:
                    constraint4[i,j,p,t]=mod.addConstr(X[i,j,p,t]*student[i]<=seat[j],name="")
                    

In [26]:
# this constraint5 is to make sure that in all time in all classroom in all day, 
#for each section, sum(X)=required sessions
constraint5={}
for i in I:
    constraint5[i]=mod.addConstr(sum(X[i,j,p,t] for j in J for p in P for t in T)==PS.get(i),name="")    

In [27]:
#constraint6: No Friday class for MW and TH section
constraint6={}
for i in I_twoday:
    constraint6[i] = mod.addConstr(sum(X[i,j,"Friday",t] for j in J for t in T) == 0) 

In [28]:
for i in I:
    print i,"\n"
    for ts in range(int(TS[i])-1):
        print T.max()-ts
        

14279 

26
25
24
23
22
21
16496 

26
25
24
15561 

26
25
14522 

26
25
24
14923 

26
25
14804 

26
25
24
16271 

26
25
24
23
22
21
15701 

26
25
14209 

26
25
14061 

26
25
24
16474 

26
25
24
14844 

26
25
24
15560 

26
25
14025 

26
25
24
14810 

26
25
24
15398 

26
25
24
23
22
21
14519 

26
25
24
16792 

26
25
24
23
22
21
16671 

26
25
24
14062 

26
25
24
14055 

26
25
24
14907 

26
25
24
16705 

26
25
24
23
22
21
14741 

26
25
24
14495 

26
25
24
14786 

26
25
24
14514 

26
25
24
14828 

26
25
24
14513 

26
25
24
14515 

26
25
24
14497 

26
25
24
14830 

26
25
24
14648 

26
25
24
14057 

26
25
24
16470 

26
25
24
15057 

26
25
24
16298 

26
25
24
23
22
21
14384 

26
25
24
66771 

26
25
16504 

26
25
24
14236 

26
25
14743 

26
25
24
15372 

26
25
24
16703 

26
25
24
23
22
21
16120 

26
25
24
23
22
16321 

26
25
24
23
22
14650 

26
25
24
15099 

26
25
24
16549 

26
25
16480 

26
25
24
14917 

26
25
15108 

26
25
24
14065 

26
25
24
14512 

26
25
24
15723 

26
25
24
23
22
14785 

26


In [29]:
#constraint7:no class can last longer than the '9PM-9.30PM' block
constraint7={}
for i in I:
    constraint7[i] = mod.addConstr(sum(X[i,j,p,T.max()-ts] for j in J for p in P for ts in range(int(TS[i])-1))<=0)
    

In [30]:
# building to classroom
ACC=[]
BRI=[]
HOH=[]
JFF=[]
JKP=[]
for j in J:
    if j[0:3]=="ACC":
        ACC.append(j)
    elif j[0:3]=="BRI":
        BRI.append(j)
    elif j[0:3]=="HOH":
        HOH.append(j)
    elif j[0:3]=="JFF":
        JFF.append(j)
    elif j[0:3]=="JKP":
        JKP.append(j)
BtoC={"ACC":ACC,"BRI":BRI,"HOH":HOH,"JFF":JFF,"JKP":JKP}

In [31]:
# constraint8 for each professor, make them teach in same building for everyday they teach
constraint8={}

for f in Prof:
    for p in P:
        for b in Building:
            courses=professor.get(f)
            classrooms=BtoC.get(b)
            for i in courses:
                for j in classrooms:
                    for t in T:
                        constraint8[f,p,b,i,j,t]=mod.addConstr(PD[f,p,b]>=X[i,j,p,t])
                        
                        
for f in Prof:
    for p in P:
        mod.addConstr(sum(PD[f,p,b] for b in Building)<=1)
        

In [32]:
# constraint9 is to minimum the day that professor show in campus
constraint9={}

for f in Prof:
    for p in P:
        for i in professor.get(f):
            constraint9[f,p,i]=mod.addConstr(sum(X[i,j,p,t] for j in J for t in T)<=D[p,f])    
            

In [33]:
#constraint10 is to maximum the prime time of classrooms used by the course schedule
constraint10={}
for i in I:
    #print i
    constraint10[i]=mod.addConstr(sum(X[i,j,p,t] for j in J for p in P for t in primetime if(t+TS[i])<=20)>=PT[i])


In [34]:
#print "number of constrints "+str(len(constraint10)+len(constraint9)+len(constraint8)+len(constraint7)+len(constraint6)+len(constraint5)+len(constraint4)+len(constraint3)+len(constraint2)+len(constraint1))

## Best solution output

In [35]:
mod.setParam("TimeLimit",600)
mod.optimize()
mod.setParam('OutputFlag',False)

print('Optimal solution:',mod.ObjVal)

Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 234315 rows, 139585 columns and 908220 nonzeros
Variable types: 0 continuous, 139585 integer (139500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [1e-03, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 203738 rows and 85132 columns (presolve time = 5s) ...
Presolve removed 214532 rows and 96034 columns (presolve time = 10s) ...
Presolve removed 222932 rows and 104434 columns
Presolve time: 14.53s
Presolved: 11383 rows, 35151 columns, 258163 nonzeros
Variable types: 0 continuous, 35151 integer (35151 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Presolve removed 2000 rows and 2000 columns
Presolved: 9383 rows, 33151 columns, 254163 nonzeros

Presolve removed 2064 rows and 2213 columns

Root simplex log...

Iteration    Objective  

In [36]:
# get the small subset of J that have classes in this classroom
J1=[]
for j in J:
    if sum(X[i,j,p,t].x for i in I for p in P for t in T)>0:
        J1.append(j)
#print J1

In [37]:
# return a blank table
def blanktable():
    T1=[]
    blank=[]
    time = datetime.timedelta(hours=8,minutes=0, seconds=0)
    for i in range(27):
        T1.append(str(time+timedelta(hours=0.5*i)))
        blank.append("")
    schedule=pd.DataFrame({"# timeslot": range(27),
                           "timeslot":T1,
                           "Monday":blank,
                           "Tuesday":blank,
                           "Wednesday":blank,
                           "Thursday":blank,
                           "Friday":blank})
    sched_cols = ['# timeslot','timeslot','Monday','Tuesday','Wednesday','Thursday','Friday']
    schedule = schedule[sched_cols]
    schedule=schedule.set_index('# timeslot')
    return schedule

In [38]:
# put the optimal solution into our beautiful schedule table
timetable={}
for j in J1:
    #print "J is "+j
    #print "schedule"+j
    schedule1=blanktable()
    for i in I:
        for p in P:
            for t in T:
                if X[i,j,p,t].x>0:
                    for a in range(int(TS[i])):
                        schedule1.loc[t+a,p]=C[i]+" / "+str(i)
                        #print schedule1
    timetable["schedule"+j]=schedule1
    #print "timetable "+"schedule"+j
    #print timetable["schedule"+j]
    

In [39]:
output=pd.ExcelWriter("schedule_2hours_sections.xlsx")
for j in J:
    if sum(X[i,j,p,t].x for i in I for p in P for t in T)>0:
        a=j+" "+str(seat[j])
        timetable["schedule"+j].to_excel(output,sheet_name=a,index=False)
output.save()

In [40]:
# make sure that every professor is assigned to one building in the day that they have class
# constraint8
for f in Prof:
    for p in P:
        for b in Building:
            if PD[f,p,b].x>0:
                print f+" "+p+" "+b+" "+str(PD[f,p,b].x)

Louk, Robert, J Monday ACC 1.0
Louk, Robert, J Tuesday ACC 1.0
Louk, Robert, J Wednesday ACC 1.0
Louk, Robert, J Thursday ACC 1.0
Louk, Robert, J Friday ACC 1.0
Wilbur, Therese, Kujawa Monday JFF 1.0
Wilbur, Therese, Kujawa Tuesday BRI 1.0
Wilbur, Therese, Kujawa Wednesday JFF 1.0
Wilbur, Therese, Kujawa Thursday BRI 1.0
Burgos, Miriam, T Monday JFF 1.0
Burgos, Miriam, T Tuesday JFF 1.0
Burgos, Miriam, T Wednesday JFF 1.0
Burgos, Miriam, T Thursday JFF 1.0
Hopkins, Merle, W Monday JKP 1.0
Hopkins, Merle, W Tuesday JFF 1.0
Hopkins, Merle, W Wednesday BRI 1.0
Hopkins, Merle, W Thursday JFF 1.0
Mukherjee, Gourab Monday JFF 1.0
Mukherjee, Gourab Tuesday JFF 1.0
Mukherjee, Gourab Wednesday JFF 1.0
Mukherjee, Gourab Thursday JFF 1.0
Mukherjee, Gourab Friday JKP 1.0
Kovacevich, Rex, Alan Monday JKP 1.0
Kovacevich, Rex, Alan Tuesday BRI 1.0
Kovacevich, Rex, Alan Wednesday BRI 1.0
Kovacevich, Rex, Alan Thursday ACC 1.0
Kovacevich, Rex, Alan Friday ACC 1.0
Ansari, Arif Monday JFF 1.0
Ansari, Ari

In [41]:
# check every professor's class day
# constraint9
for f in Prof:
    for p in P:
        if D[p,f].x>0:
            print f+" "+p+" "+str(D[p,f].x)

Louk, Robert, J Friday 1.0
Wilbur, Therese, Kujawa Tuesday 1.0
Wilbur, Therese, Kujawa Thursday 1.0
Burgos, Miriam, T Tuesday 1.0
Burgos, Miriam, T Thursday 1.0
Hopkins, Merle, W Tuesday 1.0
Hopkins, Merle, W Thursday 1.0
Mukherjee, Gourab Friday 1.0
Kovacevich, Rex, Alan Friday 1.0
Ansari, Arif Tuesday 1.0
Snyder, Kirk, Dylan Tuesday 1.0
Snyder, Kirk, Dylan Thursday 1.0
Kiddoo, Bob Monday 1.0
Kiddoo, Bob Wednesday 1.0
Swenson, Charles, W Tuesday 1.0
Swenson, Charles, W Thursday 1.0
Cavanaugh, Lisa, Ann Monday 1.0
Cavanaugh, Lisa, Ann Wednesday 1.0
Salomone, Anthony Friday 1.0
Bristow, Duke Tuesday 1.0
Bristow, Duke Thursday 1.0
Randhawa, Smrity, P Monday 1.0
Randhawa, Smrity, P Wednesday 1.0
Fast, Nathanael Monday 1.0
Coombs, Michael, Wallace Tuesday 1.0
Coombs, Michael, Wallace Thursday 1.0
Coombs, Michael, Wallace Friday 1.0
Layton, Rose, M Monday 1.0
Layton, Rose, M Wednesday 1.0
Phiroz, Zal Monday 1.0
Phiroz, Zal Wednesday 1.0
Kim, Peter, Hochin Friday 1.0
Bresnahan, Chris Friday 

In [42]:
# check the number of sessions in prime time of every section
# constraint10

for i in I:
    print str(i)+" "+str(PT[i].x)

14279 1.0
16496 2.0
15561 2.0
14522 2.0
14923 1.0
14804 1.0
16271 0.0
15701 2.0
14209 2.0
14061 2.0
16474 2.0
14844 1.0
15560 2.0
14025 2.0
14810 1.0
15398 0.0
14519 0.0
16792 1.0
16671 2.0
14062 2.0
14055 2.0
14907 2.0
16705 1.0
14741 1.0
14495 2.0
14786 2.0
14514 2.0
14828 1.0
14513 2.0
14515 2.0
14497 2.0
14830 1.0
14648 2.0
14057 0.0
16470 2.0
15057 1.0
16298 0.0
14384 0.0
66771 2.0
16504 2.0
14236 2.0
14743 1.0
15372 2.0
16703 0.0
16120 1.0
16321 1.0
14650 2.0
15099 2.0
16549 2.0
16480 2.0
14917 1.0
15108 2.0
14065 2.0
14512 2.0
15723 0.0
14785 0.0
14387 1.0
14968 2.0
15362 2.0
15343 0.0
14725 1.0
14892 1.0
15660 2.0
14920 1.0
14732 1.0
14895 1.0
14096 2.0
66750 2.0
15100 0.0
14381 2.0
14916 1.0
66746 2.0
15592 0.0
14901 2.0
14894 1.0
66781 2.0
15053 1.0
14548 2.0
66769 2.0
14026 2.0
15661 2.0
15904 2.0
16548 2.0
15440 1.0
15753 0.0


In [43]:
mod.presolve()
mod.write('check.lp')

## compare for report

In [44]:
#sum(professor's teaching days per week)/total number of professors

print "new prof's teaching rate "+str(sum(D[p,f].x for p in P for f in Prof)/len(Prof))
dayofprof=pd.DataFrame(columns=["professor","teaching day"],index=I)
data1=data[["Section","FirstInstructor","FirstDays"]]
num=0
abc=[]
for i in I:
    #data1.loc[data1.loc[:,"Section"]==i,"FirstInstructor"]
    dayofprof.loc[i,"professor"]=str(data1.loc[data1.loc[:,"Section"]==i,"FirstInstructor"].values)
    dayofprof.loc[i,"teaching day"]=str(data1.loc[data1.loc[:,"Section"]==i,"FirstDays"].values)
    #abc.append(str(data1.loc[data1.loc[:,"Section"]==i,"FirstDays"]))
numofteachday=0
for j in dayofprof["teaching day"]:
    if len(j)==7:
        numofteachday+=2
    elif len(j)==6:
        numofteachday+=1
print "origional prof's teaching rate "+str(numofteachday/len(Prof))

new prof's teaching rate 1.66666666667
origional prof's teaching rate 2


In [45]:
#capacity=pd.DataFrame({"Room":["JFF LL105","ACC310","BRI202","ACC312","JKP110","JFF236"#,"JKP102",
                               #,"JFF240","JKP104","HOH1","JKP112","JFF414","JKP210","JKP212"
 #                              ],
  #                     "Size":[149,54,42,20,77,60#,52,48,
                               #56,73,77,60,78,78
   #                           ]})
#sum(capacity["Size"])

In [46]:
# method from jonanna
#utilization rate of classroom
# data after optimization
capacity2=capacity1.set_index("Room")
allcapacity=sum(sum(X[i,j,p,t].x for i in I_oneday for p in P for t in T)*capacity2.loc[j,"Size"] for j in J
               )+sum(sum(X[i,j,p,t].x for i in I_twoday for p in P for t in T)*capacity2.loc[j,"Size"] for j in J)/2
allregcount=sum(section_info["Reg Count"])
print allregcount
print allcapacity
print "new classroom utilization rate "+str(allregcount/allcapacity)

# origional data
#data.columns=["Course","Section","FirstDays","FirstBeginTime","FirstEndTime","FirstRoom","FirstInstructor","regcount"]
allcapacity=0
allregcount=0
for i in I:
    #print i
    allregcount=allregcount+int(data.loc[data.loc[:,"Section"]==i,"regcount"].values)
    allcapacity=allcapacity+int(data.loc[data.loc[:,"Section"]==i,"Seats"].values)

print "origional classroom utilization rate "+str(float(allregcount)/allcapacity)

3150.0
5372.0
new classroom utilization rate 0.586373790022
origional classroom utilization rate 0.921157419697


In [47]:
capacity=capacity.set_index("Room")

In [48]:

# new classroom utilization rate based on classroom
one_session=[]
for i in I:
    for j in J:
        if sum(X[i,j,p,t].x for p in P for t in T)>0:
            #print j
            one_session.append(student[i]/capacity.loc[j,"Size"])
            
print sum(one_session)
print len(one_session)
print "new classroom utilization rate "+str(float(sum(one_session))/len(one_session))



#utilization rate
#sum(number of registered students)/sum(seats in the assigned classroom)
#print "new classroom utilization rate "+str(sum(section_info["Reg Count"])/sum(capacity["Size"]))


meanreg=[]
capacitysize=0
for j in J:
    cr=data.loc[data.loc[:,"FirstRoom"]==j,"regcount"]
    if len(cr)!=0:
        a=(float(sum(cr))/len(cr))/capacity.loc[j,"Size"]
        #print a
        meanreg.append((float(sum(cr))/len(cr))/capacity.loc[j,"Size"])
        #capacitysize=capacitysize+capacity.loc[j,"Size"]
        #print j
print sum(meanreg)
print len(capacity)
print "origional classroom utilization rate "+str(float(sum(meanreg))/len(capacity))



60.7018096836
85
new classroom utilization rate 0.714138937454
2.92698121984
6
origional classroom utilization rate 0.487830203306


In [49]:
#utilization rate prime time of optimized result
#sum(primetime以内的每节课时长[按分钟算])/(教室数*一天8小时*60*5天[按分钟算])
usedprimetime=sum(int(section_info.loc[section_info.loc[:,"section"]==i,"timelength"].values*30*PT[i].x) for i in I)
allprimetime=(8*60*5*len(capacity))
print "new utilization rate of prime time "+str(float(usedprimetime)/allprimetime)

#utilization rate prime time of origional data
#sum(primetime以内的每节课时长[按分钟算])/(教室数*一天8小时*60*5天[按分钟算])
#data[["FirstBeginTime","FirstEndTime"]]
begintime=datetime.time(10,0,0)
endtime=datetime.time(18,0,0)
timedata=data.loc[(data.loc[:,"FirstBeginTime"]>=begintime)&
                  (data.loc[:,"FirstEndTime"]<=endtime),["FirstBeginTime","FirstEndTime"]]
# timelength
timelength=[]
for i in timedata.index:
    time1=timedata.loc[i,"FirstEndTime"]
    time2=timedata.loc[i,"FirstBeginTime"]
    minutediff=((time1.hour*60+time1.minute)-(time2.hour*60+time2.minute)+10)
    halfhour=minutediff/30
    if minutediff%30>0:
        halfhour+=1
    timelength.append(halfhour)
    
usedprimetime=sum(timelength)*30
allprimetime=(8*60*5*len(capacity1))
print "origional utilization rate of prime time "+str(float(usedprimetime)/allprimetime)



new utilization rate of prime time 0.95625
origional utilization rate of prime time 0.342045454545


In [50]:
result=pd.DataFrame(columns=["I section","J classroom","P day of week","T time slots"])
num=0
for i in I:
    for j in J:
        for t in T:
            for p in P:
                result.loc[num,"I section"]=i
                result.loc[num,"J classroom"]=j
                result.loc[num,"P day of week"]=p
                result.loc[num,"T time slots"]=t
                num+=1
result.to_excel("saveX_new_ratio.xlsx")

KeyboardInterrupt: 

In [ ]:
section_info.to_excel("85_sestions.xlsx")

In [ ]:
capacity=pd.DataFrame({"Room":["JFF LL105","JKP110","JFF414","ACC201","JKP104","ACC312"],
                       "Size":[149,77,60,48,56,20
                              ]})
capacity.to_csv("6_classrooms.csv")